In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.



^GSPC (S&P 500 Index) : Un indice représentant les 500 plus grandes entreprises cotées sur les bourses américaines, considéré comme un indicateur clé de la performance du marché boursier américain.

^DJI (Dow Jones Industrial Average) : Un indice composé de 30 grandes entreprises américaines industrielles, souvent utilisé pour évaluer la santé de l'économie américaine.

^IXIC (NASDAQ Composite Index) : Un indice boursier qui inclut toutes les actions cotées sur le NASDAQ, connu pour sa forte concentration en entreprises technologiques.

^FTSE (FTSE 100 Index) : L'indice principal de la Bourse de Londres, comprenant les 100 plus grandes entreprises cotées sur le London Stock Exchange.

^GDAXI (DAX Performance Index) : Un indice des 30 principales entreprises cotées à la Bourse de Francfort en Allemagne.

^FCHI (CAC 40 Index) : L'indice boursier principal en France, comprenant les 40 entreprises les plus importantes de la Bourse de Paris.

^N225 (Nikkei 225) : Un indice des 225 plus grandes entreprises cotées à la Bourse de Tokyo au Japon.

^HSI (Hang Seng Index) : Un indice boursier principal de la Bourse de Hong Kong, comprenant les plus grandes entreprises de la région.

EEM (iShares MSCI Emerging Markets ETF) : Un fonds négocié en bourse qui suit l'indice MSCI Emerging Markets, représentant les marchés émergents.

BND (Vanguard Total Bond Market ETF) : Un fonds négocié en bourse qui cherche à suivre l'indice Bloomberg Barclays U.S. Aggregate Float Adjusted Index, représentant le marché obligataire américain.

In [64]:
# Définir les symboles des indices
indices = ['^GSPC', '^DJI', '^IXIC', '^FTSE', '^GDAXI', '^FCHI', '^N225', '^HSI', 'EEM', 'BND']
    # Liste des symboles d'actions que vous souhaitez récupérer
symbols = indices

# Créez une liste vide pour stocker les données
data_list = []

# Boucle pour récupérer les données pour chaque symbole
for symbol in symbols:
    # Utilisez la bibliothèque yfinance pour obtenir les données
    ticker = yf.Ticker(symbol)
    data = ticker.history(period="1y")  # Récupère les données pour la journée en cours

    # Ajoutez les données dans la liste
    data_list.append(data)

# Concaténez les données de chaque symbole en une seule DataFrame
data = pd.concat(data_list, keys=symbols, names=['Symbol'])


In [65]:
# Conserver les noms actuels de l'index multi-niveaux
index_names = data.index.names

# Supprimer les informations de fuseau horaire et conserver le niveau 'Symbol'
new_indices = pd.MultiIndex.from_tuples([(idx[0], idx[1].replace(tzinfo=None)) for idx in data.index], names=index_names)

# Assigner le nouvel index multi-niveaux au DataFrame
data.index = new_indices

In [79]:
n_unique_symboles = data.index.get_level_values('Symbol').nunique()
symboles_uniques = data.index.get_level_values('Symbol').unique()

indices = ['^GSPC', '^DJI', '^IXIC', 'EEM', 'BND','^FTSE', '^GDAXI', '^FCHI', '^N225', '^HSI']

closing_price_df= pd.DataFrame(index = data.loc['EEM'].index, columns = indices)

for symbol in indices:
    close_price = data.loc[symbol, 'Close']
    closing_price_df[symbol] = close_price.reindex(closing_price_df.index)


In [80]:
closing_price_df

,^GSPC,^DJI,^IXIC,EEM,BND,^FTSE,^GDAXI,^FCHI,^N225,^HSI
Date,,,,,,,,,,
2022-11-23,4027.260010,34194.058594,11285.320312,37.089695,70.385780,NaN,NaN,NaN,NaN,NaN
2022-11-25,4026.120117,34347.031250,11226.360352,36.913773,70.385780,7486.700195,14541.379883,6712.479980,28283.029297,17573.580078
2022-11-28,3963.939941,33849.460938,11049.500000,36.747623,70.298515,7474.000000,14383.360352,6665.200195,28162.830078,17297.939453
2022-11-29,3957.629883,33852.531250,10983.780273,37.539265,70.036591,7512.000000,14355.450195,6668.970215,28027.839844,18204.679688
2022-11-30,4080.110107,34589.769531,11468.000000,38.624104,70.579803,7573.100098,14397.040039,6738.549805,27968.990234,18597.230469
...,...,...,...,...,...,...,...,...,...,...
2023-11-16,4508.240234,34945.468750,14113.669922,39.310001,70.540001,7411.000000,15786.610352,7168.399902,33424.410156,17832.820312
2023-11-17,4514.020020,34947.281250,14125.480469,39.360001,70.639999,7504.299805,15919.160156,7233.910156,33585.199219,17454.189453
2023-11-20,4547.379883,35151.039062,14284.530273,39.849998,70.739998,7496.399902,15901.330078,7246.930176,33388.031250,17778.070312


In [81]:
def StockReturnsComputing(StockPrice):
    StockReturn = (StockPrice.diff()/StockPrice.shift(1)) * 100
    return StockReturn

# Supposons que closing_price_df_1 soit votre DataFrame original
stockReturns = StockReturnsComputing(closing_price_df)

# Affichage des rendements calculés
stockReturns


,^GSPC,^DJI,^IXIC,EEM,BND,^FTSE,^GDAXI,^FCHI,^N225,^HSI
Date,,,,,,,,,,
2022-11-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25,-0.028304,0.447366,-0.522448,-0.474316,0.000000,NaN,NaN,NaN,NaN,NaN
2022-11-28,-1.544419,-1.448656,-1.575402,-0.450101,-0.123981,-0.169637,-1.086689,-0.704356,-0.424987,-1.568494
2022-11-29,-0.159187,0.009070,-0.594776,2.154265,-0.372589,0.508429,-0.194045,0.056563,-0.479321,5.241897
2022-11-30,3.094787,2.177794,4.408498,2.889878,0.775613,0.813367,0.289715,1.043333,-0.209968,2.156318
...,...,...,...,...,...,...,...,...,...,...
2023-11-16,0.119043,-0.130725,0.069698,-0.957413,0.598971,-1.013769,0.244095,-0.571598,-0.284278,-1.361689
2023-11-17,0.128205,0.005187,0.083682,0.127192,0.141761,1.258937,0.839634,0.913876,0.481053,-2.123225
2023-11-20,0.739028,0.583043,1.125978,1.244913,0.141561,-0.105272,-0.112004,0.179986,-0.587068,1.855605


In [84]:
log_return = np.log(1 + closing_price_df.pct_change())

In [85]:
import plotly.graph_objects as go

# Création d'une figure
fig = go.Figure()

# Boucle pour ajouter les données de chaque symbole dans la figure
for symbol in stockReturns.columns:
    fig.add_trace(go.Scatter(x=stockReturns.index, y=stockReturns[symbol], mode='lines', name=symbol))

# Mise à jour du layout pour ajouter des fonctionnalités interactives
fig.update_layout(
    title='Rendements des Symboles',
    xaxis_title='Date',
    yaxis_title='Rendement',
    legend_title='Symboles',
    hovermode='x'
)

# Ajouter des boutons pour sélectionner/désélectionner tous les symboles
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=[{"visible": [True] * len(stockReturns.columns)}],
                    label="Afficher Tous",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False] * len(stockReturns.columns)}],
                    label="Masquer Tous",
                    method="update"
                )
            ]),
        )
    ]
)

# Afficher le graphique
fig.show()


# 1000usd investment viz


In [86]:
initial_investment = 1000
investment_value = initial_investment * (1 + stockReturns.fillna(0) / 100).cumprod()

In [90]:
investment_value

,^GSPC,^DJI,^IXIC,EEM,BND,^FTSE,^GDAXI,^FCHI,^N225,^HSI
Date,,,,,,,,,,
2022-11-23,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
2022-11-25,999.716956,1004.473662,994.775517,995.256839,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
2022-11-28,984.277134,989.922294,979.103800,990.777181,998.760190,998.303632,989.133113,992.956436,995.750129,984.315056
2022-11-29,982.710298,990.012085,973.280330,1012.121149,995.038916,1003.379300,987.213752,993.518079,990.977294,1035.911841
2022-11-30,1013.123091,1011.572506,1016.187373,1041.370213,1002.756567,1011.540452,990.073855,1003.883784,988.896555,1058.249394
...,...,...,...,...,...,...,...,...,...,...
2023-11-16,1119.431133,1021.974875,1250.622005,1059.863161,1002.191076,978.401328,1095.103247,1073.977758,1127.503436,1050.267976
2023-11-17,1120.866298,1022.027881,1251.668546,1061.211224,1003.611796,990.718782,1104.298110,1083.792578,1132.927323,1027.968426
2023-11-20,1129.149812,1027.986747,1265.762059,1074.422383,1005.032515,989.675836,1103.061253,1085.743252,1126.276269,1047.043462


In [87]:
import plotly.graph_objects as go

# Création d'une figure
fig = go.Figure()

# Ajout des données pour les rendements normaux
for symbol in investment_value.columns:
    fig.add_trace(go.Scatter(x=investment_value.index, y=investment_value[symbol], mode='lines', name=symbol + " (Normal)"))

# Mise à jour du layout
fig.update_layout(
    title='Valeur de l\'Investissement de 1000 Euros au Fil du Temps',
    xaxis_title='Date',
    yaxis_title='Valeur de l\'Investissement',
    legend_title='Symboles'
)

# Afficher le graphique
fig.show()


In [88]:
percentage_growth = (stockReturns.fillna(0) / 100).cumsum() * 100
percentage_growth

,^GSPC,^DJI,^IXIC,EEM,BND,^FTSE,^GDAXI,^FCHI,^N225,^HSI
Date,,,,,,,,,,
2022-11-23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-11-25,-0.028304,0.447366,-0.522448,-0.474316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-11-28,-1.572724,-1.001290,-2.097851,-0.924417,-0.123981,-0.169637,-1.086689,-0.704356,-0.424987,-1.568494
2022-11-29,-1.731910,-0.992219,-2.692626,1.229848,-0.496570,0.338792,-1.280733,-0.647794,-0.904308,3.673403
2022-11-30,1.362877,1.185574,1.715872,4.119726,0.279043,1.152159,-0.991019,0.395540,-1.114276,5.829721
...,...,...,...,...,...,...,...,...,...,...
2023-11-16,12.287925,2.905152,24.146640,7.074972,0.486481,-1.536132,10.034448,8.212119,13.124809,7.376311
2023-11-17,12.416130,2.910338,24.230322,7.202165,0.628243,-0.277195,10.874083,9.125994,13.605862,5.253086
2023-11-20,13.155158,3.493382,25.356300,8.447078,0.769803,-0.382467,10.762079,9.305980,13.018794,7.108691


In [89]:
import plotly.graph_objects as go

# Création d'une figure
fig = go.Figure()

# Ajout des données pour les rendements normaux
for symbol in percentage_growth.columns:
    fig.add_trace(go.Scatter(x=percentage_growth.index, y=percentage_growth[symbol], mode='lines', name=symbol + " (Normal)"))

# Mise à jour du layout
fig.update_layout(
    title='Croissance en Pourcentage de 1000 Euros Investis',
    xaxis_title='Date',
    yaxis_title='Croissance en Pourcentage',
    legend_title='Symboles'
)

# Afficher le graphique
fig.show()
